In [ ]:
from flask import Flask, jsonify, request, Response
from SearchEngine import SearchEngine

app = Flask(__name__)

@app.route('/', methods=['GET'])
def search():
    return se.

if __name__ == '__main__':
    se = SearchEngine()
    se.buildTries()
    app.run(debug=True, port=8886)
